## Import Necessary Modules 

In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv
import os
from langchain_groq import ChatGroq
import ipywidgets as widgets
from IPython.display import display, HTML
from langchain_core.documents import Document
from langchain_chroma import Chroma
import os
import json
import requests
from langchain_community.vectorstores import Chroma
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain_chroma import Chroma
from langchain.agents import create_tool_calling_agent
from datetime import datetime
import pytz
from langchain.memory import ConversationBufferWindowMemory
import json
from typing import List, Literal, Optional

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.documents import Document
from langchain_core.embeddings import Embeddings
from langchain_core.messages import get_buffer_string
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig
from langchain_core.tools import tool
from langchain_core.vectorstores import InMemoryVectorStore
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, MessagesState, StateGraph
from langgraph.prebuilt import ToolNode
import uuid


### Load Environment Variable

In [2]:
# Load the API key from .env file
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
SERP_API_KEY = os.getenv("SERP_API_KEY")

## Populating the Vector Database
Check populate_vector_bd.ipynb file

## Vector Configuration

In [3]:
book_persist_directory="./Dataset/books_chroma"
longterm_memory_persist_dir = "./Dataset/langgraph_conversation_chromattt"
embedder = HuggingFaceEmbeddings(model_name="BAAI/llm-embedder") # Embeddings Model

## Build Agents - Book Analysis and Internet Search

In [4]:
# ================== Book Analysis Agent ==================
class BookAnalysisAgent:
    def __init__(self):        
        # Create LangChain vectorstore
        self.vectorstore = Chroma(persist_directory=book_persist_directory, 
                collection_name="Adavance_RAG_Test",
                embedding_function=embedder)
        self.retriever = self.vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

    def answer_book(self, query):
        """Process book queries with error handling"""
        try:
            docs = self.retriever.invoke(query)
            if not docs:
                return "No relevant book passages found."
            return "\n".join([f"From {doc.metadata['title']}:\n{doc.page_content}" for doc in docs])
        except Exception as e:
            return f"Book search error: {str(e)}"

# ================== Internet Search Agent ==================
class InternetSearchAgent:
    def __init__(self):
        self.url = "https://google.serper.dev/search"
        self.headers = {
        'X-API-KEY': SERP_API_KEY,
        'Content-Type': 'application/json'
        }
        self.history = []

    def search_web(self, query):
        """Perform web search with error handling"""
        try:
            payload = json.dumps({
                "q": query,
                "location": "India",
                "gl": "in"
            })
            response = requests.post(self.url, headers=self.headers, data=payload, timeout=10)
            response.raise_for_status()
            
            result = response.json()
            self.history.append({"query": query, "result": result})
            
            # Extract and format relevant information
            if 'organic' not in result:
                return "No relevant web results found."
                
            top_results = result['organic'][:3]
            return "\n".join([f"{res['title']}: {res.get('snippet', '')}" for res in top_results])
            
        except Exception as e:
            return f"Search error: {str(e)}"


#### Initialize agents

In [5]:
# Initialize agents
book_agent = BookAnalysisAgent()
internet_agent = InternetSearchAgent()

tools = [
    Tool(
        name="Literary Analysis",
        func=book_agent.answer_book,
        description="Analysis of book themes and content"
    ),
    Tool(
        name="Web Search",
        func=internet_agent.search_web,
        description="Current information and general knowledge"
    )
]


## Memory Configuration

### Long-term memory

In [ ]:
# Define vectorstore for memories - Long-term memory using Chroma
recall_vector_store = Chroma(
            collection_name="conversation_history",
            persist_directory=longterm_memory_persist_dir,
            embedding_function=embedder
        )

In [16]:
# Get user ID from the configuration
def get_user_id(config: RunnableConfig) -> str:
    user_id = config["configurable"].get("user_id")
    if user_id is None:
        raise ValueError("User ID needs to be provided to save a memory.")
    return user_id

In [ ]:
# ================== Long-Term Memory ==================
@tool
def save_recall_memory(memory: str, config: RunnableConfig) -> str:
    """Save memory to vectorstore for later semantic retrieval."""
    user_id = get_user_id(config)
    tz_Mumbai = pytz.timezone('Asia/Kolkata')
    datetime_Mumbai = datetime.now(tz_Mumbai)
    document = Document(
        page_content=memory, id=str(uuid.uuid4()), metadata={"user_id": user_id, "Time Date": str(datetime_Mumbai)}
    )
    recall_vector_store.add_documents([document])
    return memory

@tool
def search_recall_memories(query: str, config: RunnableConfig) -> List[str]:
    """Search for relevant memories."""
    user_id = get_user_id(config)
    # Perform a similarity search in the vectorstore
    documents = recall_vector_store.similarity_search(
        query, k=3, filter={"user_id": user_id}
    )
    return [document.page_content for document in documents]

#### Test Long-term Memory

In [ ]:
# # Mock configuration
# config = {"configurable": {"user_id": "1", "thread_id": "1"}}
# # Test save
# save_output = save_recall_memory.invoke("User likes hiking and photography.", config)
# print("Save Output:", save_output)
# # Test search
# search_output = search_recall_memories.invoke("What does the user like?", config)
# print("Search Output:", search_output)

Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1


Save Output: User likes hiking and photography.
Search Output: ['User likes hiking and photography.']


### Short-Term Memory

In [21]:
# ================== Short-Term Memory ==================
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.runnables import RunnableConfig
from langchain_core.tools import tool

# Dictionary to store short-term memory instances per user
user_short_term_memories = {}

def get_user_memory(user_id: str) -> ConversationBufferWindowMemory:
    if user_id not in user_short_term_memories:
        # Each user gets a memory that remembers only the last 3 turns
        user_short_term_memories[user_id] = ConversationBufferWindowMemory(
            k=3,
            return_messages=True
        )
    return user_short_term_memories[user_id]

@tool
def save_short_term_message(message: str, is_user: bool, config: RunnableConfig) -> str:
    """Save a message (user or AI) to short-term memory for a given user."""
    user_id = get_user_id(config)
    memory = get_user_memory(user_id)
    
    if is_user:
        memory.chat_memory.add_user_message(message)
    else:
        memory.chat_memory.add_ai_message(message)

    return message

@tool
def get_short_term_messages(config: RunnableConfig) -> list:
    """Get the last 3 turns (up to 6 messages) for the user from short-term memory."""
    user_id = get_user_id(config)
    memory = get_user_memory(user_id)

    return [f"{msg.type}: {msg.content}" for msg in memory.chat_memory.messages]


#### Test Short-Term Memory

In [23]:
# Mock configuration
config = {"configurable": {"user_id": "1", "thread_id": "1"}}
# Test save
save_output = save_short_term_message.invoke("User likes hiking and photography.", True, config)
print("Save Output:", save_output)
# Test search
search_output = get_short_term_messages.invoke("What does the user like?", config)
print("Search Output:", search_output)

TypeError: BaseTool.invoke() takes from 2 to 3 positional arguments but 4 were given

## Enhance Query - Add Long-Term Memory

In [8]:
# ================== Enhanced Query Handling ==================
# Define a function to extract the content of a message
def process_query(query):
    # Get relevant long-term memories with Time of question asked
    lt_memory = []
    for doc in long_term_memory.get_relevant_memories(query):
        lt_memory.append("DateTime: "+doc.metadata['Time Date']+"\n"+doc.page_content)
    lt_context = "\n\n".join(lt_memory)
    input = f"""### Long Term Memories: {lt_context}

### User Question: {query}
"""
    # Execute agent
    result = agent_executor.invoke({"input": input})
    agent_ans = result['output'].split("Final Answer:")[1].strip()
    # Store in long-term memory
    long_term_memory.add_memory(f"User: {query}\nAgent: {agent_ans}")
    
    return result['output']

In [9]:
process_ = process_query("Who was the caption of the 2007 T20 cricket world cup winner and what was his score in final match")
print(process_)

Question: Who was the captain of the 2007 T20 cricket world cup winner and what was his score in the final match
Thought: To answer this question, I need to find information about the 2007 T20 cricket world cup, specifically the winning team and its captain, as well as the captain's performance in the final match.
Action: Web Search
Action Input: 2007 T20 cricket world cup winner captain and score in final match
Observation: The winner of the 2007 T20 cricket world cup was India, and the captain was MS Dhoni. However, the observation does not provide the score of MS Dhoni in the final match.

Thought: Since I was unable to find the score of MS Dhoni in the final match, I need to reformulate the query to get more specific information.
Action: Web Search
Action Input: MS Dhoni score in 2007 T20 world cup final match
Observation: MS Dhoni scored 0 runs in the 2007 T20 world cup final match.

Thought: I now know the final answer
Final Answer: The captain of the 2007 T20 cricket world cup w

## UI Based ChatBot AskAI

In [ ]:
# Define the chatbot response function
def chatbot_response(user_input):
    # Finally, let's invoke the chain
    response = process_query(user_input)
    return f"{response}"

# Create the chatbot UI
# Text input for user messages
user_input = widgets.Text(
    placeholder="Type your message here...",
    description="You:",
    layout=widgets.Layout(width="80%")
)

# Button to submit messages
submit_button = widgets.Button(
    description="Send",
    button_style="success"
)

# Output area for the conversation
output = widgets.Output(
    layout=widgets.Layout(),
    style={"description_width": "initial"}
)

# Function to handle button click
def on_submit_button_click(b):
    with output:
        user_message = user_input.value
        if user_message.strip():  # Check if the input is not empty
            # Display the user's message
            display(HTML(f"<strong>You:</strong> {user_message}"))
            
            # Get the chatbot's response
            bot_response = chatbot_response(user_message)

            # Extract the content within the <think> tag
            # think_content = bot_response.split('Final Answer:')[0].strip()
            think_content = bot_response

            # Extract the bot's response after the <think> tag
            answer_content = bot_response.split('Final Answer:')[1].strip()

            # Format the output
            formatted_output = f"""
            <strong>AskAI AgentExecutor Thinking:</strong><think style="font-family: 'Courier New', Courier, monospace;">
            > Entering new AgentExecutor chain...
            {think_content}
            > Finished chain.
            </think>
            <strong>AskAI Answer:</strong> {answer_content}
            """
            # formatted_output = f"""<strong>AskAI Answer:</strong> {answer_content}"""
            formatted_output_html = formatted_output.replace("\n", "<br>")
            # Display the bot's response
            display(HTML(f"{formatted_output_html}"))
            display(HTML("<br>"))
            # Clear the input box
            user_input.value = ""
        else:
            display(HTML("<em>Please enter a message.</em>"))

# Attach the function to the button's click event
submit_button.on_click(on_submit_button_click)

# Arrange the widgets vertically
chatbot_ui = widgets.VBox([user_input, submit_button, output])

# Display the chatbot UI
display(chatbot_ui)

### Some Example Question and Answer

---
- You: Hi, I am Saurabh.
- AskAI Answer: Hi Saurabh, how are you today?
---
- You: I love pizza
- AskAI Answer: Pizza is a delicious and versatile food, what's your favorite type of pizza or topping, Saurabh?
---
- You:  yes -- pepperoni!
- AskAI Answer: Pepperoni pizza is a classic favorite, do you prefer a thin crust, thick crust, or something else, and do you like to add any other toppings to your pepperoni pizza, Saurabh?
---
- You: I also just moved to Bangalore.
- AskAI Answer: Bangalore is a vibrant city with a lot to offer, how are you finding the city so far, and have you had a chance to explore any of its popular spots, Saurabh?
---
- You: where should i go for dinner?
- AskAI Answer: You could try Toit, a popular spot for pizza and craft beer, or MTR, a well-known restaurant for South Indian cuisine, both of which are highly rated in Bangalore, Saurabh.
---
- You: what's the address for Toit in Bangalore?
- AskAI Answer: Toit has multiple locations in Bangalore, including Indiranagar, Koramangala, and Sarjapur Road, with addresses such as 298, Nambiar Building, 100 Feet Road, Indiranagar, and 65, 1st Block, Jyoti Nivas College Road, Koramangala 5th Block, Saurabh.
---
- You: Who murdered Megan Hipwell?
- AskAI Answer: Tom Watson, the husband of Rachel Watson, is the one who murdered Megan Hipwell in the novel "The Girl on the Train" by Paula Hawkins.
---
- You: What last 2 questions did I ask?
- AskAI Answer: Your last two questions were "what's the address for Toit in Bangalore?" and "Who murdered Megan Hipwell?"
---
- You: How many childern do Verity and Jeremy have and tell me their children names?
- AskAI Answer: Verity and Jeremy have two children, a boy named Jeremy Jr. and a girl named Emma.
---
- You: Wrong answer, Verity and Jeremy have three children. Their daughters are named Chastin and Harper, and they also have a son named Crew.
- AskAI Answer: Verity and Jeremy have three children, their daughters are named Chastin and Harper, and they also have a son named Crew.


## Testing - Not Important
If you are interested look for knowledge

In [90]:
from langgraph.graph import StateGraph, END

# Define the structure of the state
from typing import TypedDict

class ChatState(TypedDict):
    messages: list[str]

# Define a simple function to add a message
def add_user_message(state: ChatState) -> ChatState:
    state["messages"].append("User: Hello")
    return state

def reply_with_bot(state: ChatState) -> ChatState:
    state["messages"].append("Bot: Hi there!")
    return state

# Create a graph
builder = StateGraph(ChatState)
builder.add_node("add_user", add_user_message)
builder.add_node("reply", reply_with_bot)

builder.set_entry_point("add_user")
builder.add_edge("add_user", "reply")
builder.add_edge("reply", END)

# Compile the graph
graph = builder.compile()

# Run it with initial state
result = graph.invoke({"messages": []})
print(result)


{'messages': ['User: Hello', 'Bot: Hi there!']}


In [92]:
from typing import Annotated, TypedDict
from langgraph.graph import StateGraph, add_messages

class MyState(TypedDict):
    messages: Annotated[list, add_messages]

def node_1(state: MyState):
    return {"messages": ["User: Hello"]}

def node_2(state: MyState):
    return {"messages": ["Bot: Hi!"]}

builder = StateGraph(MyState)
builder.add_node("user", node_1)
builder.add_node("bot", node_2)

builder.set_entry_point("user")
builder.add_edge("user", "bot")
builder.add_edge("bot", END)

graph = builder.compile()
out = graph.invoke({"messages": []})
print(out)


{'messages': [HumanMessage(content='User: Hello', additional_kwargs={}, response_metadata={}, id='a41a3a7f-2343-4a76-a307-b319c55f797b'), HumanMessage(content='Bot: Hi!', additional_kwargs={}, response_metadata={}, id='05e3ca78-b0af-4de7-8225-748fd11c4aa5')]}


In [96]:
from typing import Annotated, Sequence
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langgraph.graph import StateGraph, add_messages
from typing_extensions import TypedDict

class ChatState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

# Node to add a user message
def add_user_message(state: ChatState) -> dict:
    return {
        "messages": [HumanMessage(content="Hey, what's up?")]
    }

# Node to simulate LLM response
def add_bot_response(state: ChatState) -> dict:
    return {
        "messages": [AIMessage(content="Not much! How can I help?")]
    }

builder = StateGraph(ChatState)
builder.add_node("user", add_user_message)
builder.add_node("bot", add_bot_response)

builder.set_entry_point("user")
builder.add_edge("user", "bot")
builder.add_edge("bot", END)

graph = builder.compile()
out = graph.invoke({"messages": []})
print(out)
for msg in out["messages"]:
    print(f"{msg.type.capitalize()}: {msg.content}")
    
out = graph.invoke({"messages": []})
print(out)
for msg in out["messages"]:
    print(f"{msg.type.capitalize()}: {msg.content}")


{'messages': [HumanMessage(content="Hey, what's up?", additional_kwargs={}, response_metadata={}, id='094f840c-b255-45e1-b128-701fb417fe76'), AIMessage(content='Not much! How can I help?', additional_kwargs={}, response_metadata={}, id='b5a69744-9e90-42c3-89df-2feada19262c')]}
Human: Hey, what's up?
Ai: Not much! How can I help?
{'messages': [HumanMessage(content="Hey, what's up?", additional_kwargs={}, response_metadata={}, id='e0407943-47d4-4e31-a175-699edf595ba7'), AIMessage(content='Not much! How can I help?', additional_kwargs={}, response_metadata={}, id='1919b4d6-3bff-4b15-9f59-def830025005')]}
Human: Hey, what's up?
Ai: Not much! How can I help?


In [25]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import AnyMessage,add_messages
from typing import Annotated,List
from typing_extensions import TypedDict

class State(TypedDict):
    messages:Annotated[List[AnyMessage],add_messages]

graph_builder = StateGraph(State)


def ChatNode(state: State) -> State:
    system_message = "You are an assistant"
    state["messages"] = "Hi"
    return state

graph_builder.add_node("chatnode", ChatNode)
graph_builder.add_edge(START, "chatnode")
graph_builder.add_edge("chatnode", END)
graph = graph_builder.compile(checkpointer=MemorySaver())